# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [22]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [6]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x
% pip install tensorflow-gpu==2.1

     |████████████████████████████████| 421.8MB 23kB/s 
     |████████████████████████████████| 3.9MB 34.6MB/s 
     |████████████████████████████████| 450kB 26.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=2b7d5c0b414d089da34a7fe90391568acbd13acc6193705550b8892093f795df
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uninstalling t

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [7]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 276kB 7.2MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=a15b423cd5c53d6b09a9d9d936f503b1485ccb75fc2c1f735aba59554b8b04c5
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [0]:
import numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers, metrics
from jarvis.train import datasets, custom
from jarvis.train.box import BoundingBox

# Data

As in the tutorial, data for this assignment will consist of prostate MRI exams. In prior work, an algorithm was created to separate out different MRI sequences. In this current assignment, only T2-weighted images (isolated using the prior algorithm) will be used for segmentation. In prostate imaging, the T2-weighted sequence captures the greatest amount of anatomic detail and is thus ideal for delineation of prostate gland structures.

The following line of code will download the dataset (if not already present) and prepare the required generators:

In [9]:
# --- Download dataset
datasets.download(name='mr/prostatex-seg')

# --- Prepare generators
configs = {'batch': {'size': 12}}
gen_train, gen_valid, client = datasets.prepare(name='mr/prostatex-seg', configs=configs, keyword='box')



[ 2020-05-15 01:39:28 ] [====================] 100.000% : Extracting archive (0001381 / 0001381) 

# Training

In this assignment we will train a box localization network for prostate segmentation.

### Define box parameters

Use the following cell block to define your `BoundingBox` object as discussed in the tutorial. Feel free to optimize hyperparameter choices for grid size, anchor shapes, anchor aspect ratios, and anchor scales: 

In [0]:
# --- Create BoundingBox
bb = BoundingBox(
    image_shape=(256, 256),
    classes=1,
    c=[3, 4, 5], #refers to c1, c2, c3, how many ancher points. 
    anchor_shapes=[16, 32, 64], #box sizes of c
    anchor_scales=[0, 1, 2], #multiplication of height and width
    anchor_ratios=[0.5, 1, 2], # ratio of height and width
    iou_upper=0.7,
    iou_lower=0.35)

# c = 3 match with anchor_shape 16, with anchor scales [0,1,2] and anchor_ratios = [0.5,1,2]
# c = 4 match with anchor_shape 32, with anchor scales [0,1,2] and anchor_ratios = [0.5,1,2]

### Define inputs

Use the following cell block to define the nested generators needed to convert raw masks into bounding box ground-truth predictions:

In [0]:
gen_train, gen_valid = client.create_generators()
gen_train, gen_valid = bb.create_generators(gen_train, gen_valid, msk='prostate')
inputs = client.get_inputs(Input)
inputs = bb.get_inputs(inputs, Input)

In [0]:
# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same'}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))

# --- Expanding 
# --- Define zoom
zoom = lambda x : layers.UpSampling3D(
    size=(1, 2, 2))(x)

# --- Define 1 x 1 x 1 projection
proj = lambda filters, x : layers.Conv3D(
    filters=filters,
    strides=1,
    kernel_size=(1, 1, 1),
    padding='same',
    kernel_initializer='he_normal')(x)

In [0]:
# --- Define contracting layers
l1 = conv1(4, inputs['dat'])
l2 = conv1(8, conv2(8, l1))
l3 = conv1(16, conv2(16, l2))
l4 = conv1(32, conv2(32, l3))
l5 = conv1(64, conv2(64, l4))
l6 = conv1(96, conv2(96, l5))
l7 = proj(96, l6)
l8 = conv1(96, zoom(l7) + proj(96, l5))
l9 = conv1(96, zoom(l8) + proj(96, l4))


### Define the model

Use the following cell block to define your feature pyramid network backbone and RetinaNet classification / regression networks:

In [0]:
# --- Determine filter sizes
logits = {}
K = 1
A = 9

Constant = 96

# --- C2
c3_cls = conv1(Constant, conv1(Constant, l9))
c3_reg = conv1(Constant, conv1(Constant, l9))
logits['cls-c3'] = layers.Conv3D(filters=(A * K), name='cls-c3', **kwargs)(c3_cls)
logits['reg-c3'] = layers.Conv3D(filters=(A * 4), name='reg-c3', **kwargs)(c3_reg)

# --- C3
c4_cls = conv1(Constant, conv1(Constant, l8))
c4_reg = conv1(Constant, conv1(Constant, l8))
logits['cls-c4'] = layers.Conv3D(filters=(A * K), name='cls-c4', **kwargs)(c4_cls)
logits['reg-c4'] = layers.Conv3D(filters=(A * 4), name='reg-c4', **kwargs)(c4_reg)

# --- C4
c5_cls = conv1(Constant, conv1(Constant, l7))
c5_reg = conv1(Constant, conv1(Constant, l7))
logits['cls-c5'] = layers.Conv3D(filters=(A * K), name='cls-c5', **kwargs)(c5_cls)
logits['reg-c5'] = layers.Conv3D(filters=(A * 4), name='reg-c5', **kwargs)(c5_reg)

model = Model(inputs=inputs, outputs=logits)

### Compile the model

Use the following cell block to compile your model. Recall the following requirements as described in the tutorial:

* use of a focal sigmoid (binary) cross-entropy loss function for regression
* use of a Huber loss function for classification
* use of masked loss functions to ensure only relevant examples are used for training
* use of appropriate metrics to track algorithm training

In [0]:
def focal_sigmoid_ce(weights=1.0, scale=1.0, gamma=2.0, alpha=0.25):
    """
    Method to implement focal sigmoid (binary) cross-entropy loss
 
    """
    def focal_sigmoid_ce(y_true, y_pred):
 
        # --- Calculate standard cross entropy with alpha weighting
        loss = tf.nn.weighted_cross_entropy_with_logits(
            labels=y_true, logits=y_pred, pos_weight=alpha)
 
        # --- Calculate modulation to pos and neg labels 
        p = tf.math.sigmoid(y_pred)
        modulation_pos = (1 - p) ** gamma
        modulation_neg = p ** gamma
 
        mask = tf.dtypes.cast(y_true, dtype=tf.bool)
        modulation = tf.where(mask, modulation_pos, modulation_neg)
 
        return tf.math.reduce_sum(modulation * loss * weights * scale)
 
    return focal_sigmoid_ce

In [0]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'cls-c3': custom.focal_sigmoid_ce(inputs['cls-c3-msk']),
        'cls-c4': custom.focal_sigmoid_ce(inputs['cls-c4-msk']),
        'cls-c5': custom.focal_sigmoid_ce(inputs['cls-c5-msk']),
        'reg-c3': custom.sl1(inputs['reg-c3-msk']),
        'reg-c4': custom.sl1(inputs['reg-c4-msk']),
        'reg-c5': custom.sl1(inputs['reg-c5-msk'])
        },
    metrics={
        'cls-c3': [custom.sigmoid_ce_sens(), custom.sigmoid_ce_ppv()],
        'cls-c4': [custom.sigmoid_ce_sens(), custom.sigmoid_ce_ppv()],
        'cls-c5': [custom.sigmoid_ce_sens(), custom.sigmoid_ce_ppv()]},
    experimental_run_tf_function=False)

### Train the model

Use the following cell block to train your model. **Note**: it is recommended to train for at least 10,000 iterations for convergence.

In [17]:
client.load_data_in_memory()

[ 2020-05-15 01:39:44 ] [====================] 100.000% : Iterating | 000342    

In [18]:
# --- Train model
model.fit(
    x=gen_train, 
    steps_per_epoch=500, 
    epochs=100,
    validation_data=gen_valid,
    validation_steps=500,
    validation_freq=4,
    use_multiprocessing=True)

Epoch 1/100
500/500 [==============================] - 87s 173ms/step - loss: 0.0300 - cls-c3_loss: 0.0109 - cls-c4_loss: 0.0104 - cls-c5_loss: 0.0087 - reg-c3_loss: 3.6946e-06 - reg-c4_loss: 1.2324e-05 - reg-c5_loss: 4.5786e-05 - cls-c3_sigmoid_ce_sens: 0.0097 - cls-c3_sigmoid_ce_ppv: 3.4262e-06 - cls-c4_sigmoid_ce_sens: 0.0158 - cls-c4_sigmoid_ce_ppv: 2.2947e-04 - cls-c5_sigmoid_ce_sens: 0.0409 - cls-c5_sigmoid_ce_ppv: 0.0670
Epoch 2/100
500/500 [==============================] - 69s 137ms/step - loss: 6.2765e-04 - cls-c3_loss: 1.6465e-04 - cls-c4_loss: 2.2410e-04 - cls-c5_loss: 2.3024e-04 - reg-c3_loss: 1.5095e-06 - reg-c4_loss: 2.1143e-06 - reg-c5_loss: 5.0359e-06 - cls-c3_sigmoid_ce_sens: 0.0000e+00 - cls-c3_sigmoid_ce_ppv: 0.0000e+00 - cls-c4_sigmoid_ce_sens: 9.5556e-04 - cls-c4_sigmoid_ce_ppv: 0.0030 - cls-c5_sigmoid_ce_sens: 0.2005 - cls-c5_sigmoid_ce_ppv: 0.2821
Epoch 3/100
500/500 [==============================] - 69s 139ms/step - loss: 3.3357e-04 - cls-c3_loss: 7.2298e-05 -

# Evaluation

Based on the tutorial discussion, use the following cells to calculate model performance. The following metrics should be calculated:

* median IoU
* 25th percentile IoU
* 75th percentile IoU

As in prior assignments, accuracy is determined on a patient by patient (volume by volume) basis, so please calculate the IoU prediction values for each 3D volume (not slice-by-slice).

### Performance

The following minimum performance metrics must be met for full credit:

* median IoU: >0.50
* 25th percentile IoU: >0.40
* 75th percentile IoU: >0.60

In [19]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)
test_train, test_valid = bb.create_generators(test_train, test_valid, msk='prostate')

ious = {
    'med': [],
    'p25': [],
    'p75': []}

for x, y in test_train:
    
    # --- Predict
    box = model.predict(x)
    if type(box) is list:
        box = {k: l for k, l in zip(model.output_names, box)}
        
    # --- Convert predictions to anchors
    anchors_pred, _ = bb.convert_box_to_anc(box)
    
    # --- Convert ground-truth to anchors
    anchors_true, _ = bb.convert_box_to_anc(y)
    
    # --- Calculate IoUs
    curr = []
    for pred, true in zip(anchors_pred, anchors_true):
        for p in pred:
            iou = bb.calculate_ious(box=p, anchors=true)
            if iou.size > 0:
                curr.append(np.max(iou))
            else: 
                curr.append(0)
    
    if len(curr) == 0:
        curr = [0]
        
    ious['med'].append(np.median(curr))
    ious['p25'].append(np.percentile(curr, 25))
    ious['p75'].append(np.percentile(curr, 75))
    
ious = {k: np.array(v) for k, v in ious.items()}

[ 2020-05-15 04:00:40 ] [====================] 100.000% : Iterating | 000273    

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort Dice score statistics. There is no need to submit training performance accuracy.

In [20]:
df = pd.DataFrame(index=np.arange(ious['med'].size))
df['iou_median'] = ious['med']
df['iou_p-25th'] = ious['p25']
df['iou_p-75th'] = ious['p75']

# --- Print accuracy
print(df['iou_median'].median())
print(df['iou_p-25th'].median())
print(df['iou_p-75th'].median())

0.7227999567985535
0.6382439583539963
0.9741638600826263


In [0]:
from google.colab import drive
drive.mount('/content/drive')